In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
import scipy as sp
import warnings
import matplotlib.pyplot as plt
from theano import tensor as tt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

az.style.use('arviz-darkgrid')
warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/benjaminwee/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Medium

## 14M2

Working through pymc3 devs solutions

In [18]:
df = pd.read_csv("/Users/benjaminwee/Documents/courses/resources/Rethinking/Data/milk.csv", sep=";")
df.loc[:, "neocortex.prop"] = df["neocortex.perc"] / 100
df.loc[:, "logmass"] = np.log(df["mass"])
df.head()

,clade,species,kcal.per.g,perc.fat,perc.protein,perc.lactose,mass,neocortex.perc,neocortex.prop,logmass
0,Strepsirrhine,Eulemur fulvus,0.49,16.60,15.42,67.98,1.95,55.16,0.5516,0.667829
1,Strepsirrhine,E macaco,0.51,19.27,16.91,63.82,2.09,NaN,NaN,0.737164
2,Strepsirrhine,E mongoz,0.46,14.11,16.85,69.04,2.51,NaN,NaN,0.920283
3,Strepsirrhine,E rubriventer,0.48,14.91,13.18,71.91,1.62,NaN,NaN,0.482426
4,Strepsirrhine,Lemur catta,0.60,27.28,19.50,53.22,2.19,NaN,NaN,0.783902


In [19]:
df["logmass"].values

array([ 0.66782937,  0.73716407,  0.92028275,  0.48242615,  0.78390154,
        1.65822808,  1.68082791,  0.92028275, -0.34249031, -0.38566248,
       -2.12026354, -0.75502258, -1.13943428, -0.51082562,  1.24415459,
        0.43825493,  1.95727391,  1.17557333,  2.07191328,  2.50959926,
        2.02683159,  1.68082791,  2.37211116,  3.56896916,  4.37487613,
        4.58210625,  3.70721041,  3.49983535,  4.00642368])

In [23]:
# prep data
neocortex = df["neocortex.prop"].values
logmass = df["logmass"].values
kcal = df["kcal.per.g"].values

# masked array for missing values
mask = np.isfinite(neocortex)
neocortex[~mask] = -999 # Replace NAs with 999
neocortex = np.ma.masked_values(neocortex, value=-999) # "Masked values" are the missing values in the model

In [25]:
with pm.Model() as m6_11_full:
    a = pm.Normal("a", mu=0.0, sd=100.0)
    sigma = pm.Exponential("sigma", 1.0)

    # need neocortex even though not used, because pymc can't compare models with different numbers of observed RVs
    # This chunk of code used for specifying the model of missing values
    mu_N = pm.Normal("mu_N", 0.5, 1.0) # Priors
    sigma_N = pm.Exponential("sigma_N", 1.0)
    neocortex_ = pm.Normal("neocortex", mu_N, sigma_N, observed=neocortex)

    mu = a

    kcal_ = pm.Normal("kcal", mu, sigma, observed=kcal)

    trace_m6_11_full = pm.sample(1000, tune=1000, random_seed=42, cores=2)

with pm.Model() as m6_12_full:
    a = pm.Normal("a", mu=0.0, sd=100.0)
    bn = pm.Normal("bn", mu=0.0, sd=1.0)
    sigma = pm.Exponential("sigma", 1.0)

    mu_N = pm.Normal("mu_N", 0.5, 1.0)
    sigma_N = pm.Exponential("sigma_N", 1.0)
    neocortex_ = pm.Normal("neocortex", mu_N, sigma_N, observed=neocortex)
    
    # neocortex_ now added to the linear model
    mu = a + bn * neocortex_

    kcal_ = pm.Normal("kcal", mu, sigma, observed=kcal)

    trace_m6_12_full = pm.sample(
        1000,
        tune=2000,
        random_seed=42,
        cores=2,
        nuts_kwargs={"target_accept": 0.9},
    )

with pm.Model() as m6_13_full:
    a = pm.Normal("a", mu=0.0, sd=100.0)
    bm = pm.Normal("bm", mu=0.0, sd=1.0)
    sigma = pm.Exponential("sigma", 1.0)

    mu_N = pm.Normal("mu_N", 0.5, 1.0)
    sigma_N = pm.Exponential("sigma_N", 1.0)
    neocortex_ = pm.Normal("neocortex", mu_N, sigma_N, observed=neocortex)

    mu = a + bm * logmass

    kcal_ = pm.Normal("kcal", mu, sigma, observed=kcal)

    trace_m6_13_full = pm.sample(1000, tune=1000, random_seed=42, cores=2)

with pm.Model() as m6_14_full:
    a = pm.Normal("a", mu=0.0, sd=100.0)
    bn = pm.Normal("bn", mu=0.0, sd=1.0)
    bm = pm.Normal("bm", mu=0.0, sd=1.0)
    sigma = pm.Exponential("sigma", 1.0)

    mu_N = pm.Normal("mu_N", 0.5, 1.0)
    sigma_N = pm.Exponential("sigma_N", 1.0)
    neocortex_ = pm.Normal("neocortex", mu_N, sigma_N, observed=neocortex)

    mu = a + bn * neocortex_ + bm * logmass

    kcal_ = pm.Normal("kcal", mu, sigma, observed=kcal)

    trace_m6_14_full = pm.sample(
        1000,
        tune=2000,
        random_seed=42,
        cores=2,
        nuts_kwargs={"target_accept": 0.9},
    )

/Users/benjaminwee/anaconda/lib/python3.6/site-packages/pymc3/model.py:1331: UserWarning: Data in neocortex contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, UserWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [neocortex_missing, sigma_N, mu_N, sigma, a]
Sampling 2 chains: 100%|██████████| 4000/4000 [00:03<00:00, 1067.20draws/s]
/Users/benjaminwee/anaconda/lib/python3.6/site-packages/pymc3/model.py:1331: UserWarning: Data in neocortex contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, UserWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [neocortex_missing, sigma_N, mu_N, sigma, bn, a]
Sampling 2 chains: 100%|██████████| 6000/6000 [00:26<00:00, 226.61draws/s]
/Users/benjaminwee/

In [31]:
comp_df_full = pm.compare(
    {
        m6_11_full: trace_m6_11_full,
        m6_12_full: trace_m6_12_full,
        m6_13_full: trace_m6_13_full,
        m6_14_full: trace_m6_14_full,
    }
)

comp_df_full.loc[:, "model"] = pd.Series(
    ["m6_11_full", "m6_12_full", "m6_13_full", "m6_14_full"]
)
comp_df_full = comp_df_full.set_index("model")
comp_df_full

/Users/benjaminwee/anaconda/lib/python3.6/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)


,WAIC,pWAIC,dWAIC,weight,SE,dSE,var_warn
model,,,,,,,
m6_14_full,-72.54,6.31,0,1,9.32,0,1
m6_13_full,-66.84,3.71,5.71,0,9.62,2.02,1
m6_11_full,-65.32,3.07,7.22,0,9.55,3.79,1
m6_12_full,-64.22,4.05,8.32,0,9.49,4.02,1


In [ ]:
# You can replace simple imputation model with a linear model of the mean using the other predictors using this code:

an = pm.Normal("an", mu=0.5, sd=1.0)
gm = pm.Normal("gm", mu=0.0, sd=1.0)
mu_N = an + gm * logmass
sigma_N = pm.Exponential("sigma_N", 1.0)
neocortex_ = pm.Normal("neocortex", mu_N, sigma_N, observed=neocortex)

## 14M3

In [33]:
d = pd.read_csv("/Users/benjaminwee/Documents/courses/resources/Rethinking//Data/WaffleDivorce.csv", ";")
d["log_population"] = np.log(d["Population"])

div_obs = d["Divorce"].values
div_sd = d["Divorce SE"].values
A = d["MedianAgeMarriage"].values
R = d["Marriage"].values
N = len(d)

In [ ]:
# Divorce model with measurement error from the book:

with pm.Model() as m_14_1:

    a = pm.Normal("a", 0, 10.0)
    bA = pm.Normal("bA", 0, 1)
    bR = pm.Normal("bR", 0, 1)

    mu = a + bA*A + bR*R
    sigma = pm.Exponential("sigma", 1)
    
    # Likelihood
    div_est = pm.Normal("div_est", mu, sigma, shape=N)
    
    # Estimation with measurement error
    # Likelihood estimate a "prior" for the "true" value
    # div_sd is data
    div_obs_ = pm.Normal("div_obs", div_est, div_sd, observed=div_obs)

    start = dict(div_est=div_obs)
    trace_14_1 = pm.sample(
        1000,
        tune=3000,
        cores=2,
        start=start,
        random_seed=42,
        nuts_kwargs=dict(target_accept=0.95),
    )
    
# Divorce model with measurement error from the book:

with pm.Model() as m_14_2:

    a = pm.Normal("a", 0, 10.0)
    bA = pm.Normal("bA", 0, 1)
    bR = pm.Normal("bR", 0, 1)

    mu = a + bA*A + bR*R
    sigma = pm.Exponential("sigma", 1)
    
    # Likelihood
    div_est = pm.Normal("div_est", mu, sigma, shape=N)
    
    # Estimation with measurement error
    # Likelihood estimate a "prior" for the "true" value
    # div_sd is data
    div_obs_ = pm.Normal("div_obs", div_est, div_sd * 2, observed=div_obs)

    start = dict(div_est=div_obs)
    trace_14_2 = pm.sample(
        1000,
        tune=3000,
        cores=2,
        start=start,
        random_seed=42,
        nuts_kwargs=dict(target_accept=0.95),
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [div_est, sigma, bR, bA, a]
Sampling 2 chains: 100%|██████████| 8000/8000 [02:07<00:00, 62.60draws/s]
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [div_est, sigma, bR, bA, a]
Sampling 2 chains: 100%|██████████| 8000/8000 [1:13:06<00:00,  1.82draws/s]      
There were 23 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 994 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.03788827724961276, but should be close to 0.95. Try to increase the number of tuning steps.
The gelman-rubin statistic

In [ ]:
az.summary(
    trace_14_1, credible_interval=0.89, round_to=2, var_names=["a", "bA", "bR", "sigma"]
)

,mean,sd,hpd_5.5%,hpd_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a,20.79,6.35,10.79,30.46,0.22,0.15,847.30,847.30,856.96,1181.06,1.0
bA,-0.53,0.21,-0.85,-0.22,0.01,0.00,918.45,918.45,919.51,1185.00,1.0
bR,0.13,0.07,0.01,0.25,0.00,0.00,967.52,933.22,961.43,1374.03,1.0
sigma,1.11,0.19,0.80,1.41,0.01,0.01,687.93,687.93,686.31,1223.30,1.0


In [ ]:
az.summary(
    trace_14_2,
    credible_interval=0.89,
    round_to=2,
    var_names=["a", "bA", "bR", "sigma"],
)

,mean,sd,hpd_5.5%,hpd_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a,22.01,5.69,9.81,26.63,2.27,1.69,6.27,6.27,7.85,58.00,1.85
bA,-0.67,0.20,-0.81,-0.29,0.10,0.08,4.12,3.81,5.01,179.77,1.85
bR,0.25,0.07,0.12,0.35,0.02,0.01,15.26,15.26,13.59,48.64,1.80
sigma,0.22,0.23,0.04,0.51,0.13,0.10,3.14,3.14,2.80,10.78,1.95


## 14H3

In [ ]:
x_cc = np.random.randn(10)
y_cc = np.random.normal(loc=x_cc)
x = np.concatenate((x_cc, np.NaN), axis=None)
y = np.concatenate((y_cc, 100.0), axis=None) 

# masked array for missing values
mask = np.isfinite(x)
x[~mask] = -999
x = np.ma.masked_values(x, value=-999)

In [ ]:
with pm.Model() as m_cc:
    a = pm.Normal("a", 0, 100)
    b = pm.Normal("b", 0, 100)

    mu = a + b * x_cc
    sigma = pm.HalfCauchy("sigma", beta=1)

    y_obs = pm.Normal("y_obs", mu=mu, sd=sigma, observed=y_cc)

    trace_cc = pm.sample(1000, tune=2000, cores=2, random_seed=RANDOM_SEED)

az.summary(trace_cc, credible_interval=0.89, round_to=2)



In [ ]:
with pm.Model() as m_full:
    a = pm.Normal("a", 0, 100)
    b = pm.Normal("b", 0, 100)
    
    # Prior centred around 0 for missing value for a corresponding y value of 100 forces the slope to be 
    x_est = pm.Normal("x_est", mu=0, sd=1, observed=x)
    mu = a + b * x_est
    sigma = pm.HalfCauchy("sigma", beta=1)

    y_obs = pm.Normal("y_obs", mu=mu, sd=sigma, observed=y)

    trace_full = pm.sample(1000, cores=2, random_seed=RANDOM_SEED)

post_full = pm.trace_to_dataframe(trace_full)
az.summary(trace_full, credible_interval=0.89, round_to=2)